# Prepare environment

In [1]:
####
DPATH_PROJECT = "/gladstone/alexanian/datasets-online/VX_projects/Project_CycsteinProteomics"
DPATH_EXPERIMENT = f"{DPATH_PROJECT}/Experiment_CysteinEnrichment"
ID_IPYNB = "001"
####

DPATH_DATA = "{}/Data".format(DPATH_EXPERIMENT)
DPATH_RESULTS="{}/Results".format(DPATH_EXPERIMENT)
DPATH_TOOL="{}/Code/tools".format(DPATH_EXPERIMENT)
DPATH_TMP="{}/tmp".format(DPATH_EXPERIMENT)

In [2]:
import os, sys, gc, importlib
import subprocess
from datetime import datetime

import numpy as np
import pandas as pd
import dask.dataframe as dd
import anndata as ad
import pickle

from glob import glob
from io import StringIO

import re
import itertools
from collections import Counter
from collections import OrderedDict

import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import AlignIO
from Bio.Align.Applications import ClustalwCommandline
from Bio import motifs
from fuc import pybed

import math
import scipy as sp
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from scipy.stats import false_discovery_control
from scipy.stats import zscore
from anndata import AnnData
import scanpy as sc
from scipy.stats import ttest_ind

from Bio import Entrez, SeqIO
from anndata import AnnData
import scanpy as sc
import scvelo as scv

import matplotlib
from matplotlib import cm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
print(pd.__name__, pd.__version__)
print(ad.__name__, ad.__version__)
print(sp.__name__, sp.__version__)
print(matplotlib.__name__, matplotlib.__version__)

pandas 2.1.1
anndata 0.10.7
scipy 1.11.3
matplotlib 3.6.2


In [4]:
%load_ext autoreload

In [5]:
%reload_ext autoreload

In [6]:
%autoreload 2

In [7]:
os.makedirs(f"{DPATH_RESULTS}/{ID_IPYNB}", exist_ok=True)
os.makedirs(f"{DPATH_DATA}/{ID_IPYNB}", exist_ok=True)
os.makedirs(f"{DPATH_TMP}/{ID_IPYNB}", exist_ok=True)

# Import Datasets

"Final_Data" sheet
Summary: This is the data that made it through some of the filtering steps that we included in the data processing software I am using. Essentially, we ask that a peptide be identified in 3/4 MS runs included for the experiment. In some cases, this filter is applied during the processing steps such that all peptides in the "Final_Data" sheet meet this 3/4 requirement. In other cases, the filter was not applied, and we will need to apply that filter ourselves here (apologies for the inconsistencies here!). In any case, it would be good practice to make sure that this filter is applied to all of them before moving forward with any analysis. If this "3/4 requirement" isn't making a ton of sense yet, don't worry. I will go into that in more detail below.

Column A: Identifier
This column contains the ID for the specific cysteine we are detecting. This contains the uniprot protein ID for the protein that the cysteine-containing peptide is coming from, along with the cysteine residue position # in that protein. So if the protein is BTK, which has an accession number of Q06187, and the cysteine being detected on BTK is in position 481 of the protein, the  "Identifier" will be Q06187_C481.

Column B: AltIdentifier
This is the same as column A, but instead of the uniprot accession number being used, it's the protein name. So in our BTK example above, here it would be listed as BTK_C481.

Column C: Modified_Peptide
This is the sequence of the peptide that contains the cysteine being detected. Importantly, the cysteine that was tagged with our chemical probe is flagged with an asterisk. In some instances, residues here are followed by a number within brackets (i.e., [0.9840]). This is just the mass of a modification found on that residue.

Columns D — G: Log2R/../../../../MS_RUN_ID
These columns contain the Log2 Heavy/Light ratios for the peptides. We have biological duplicates of each cell line in the NCI60 (A and B). For each sample, we run it twice on the MS (1 and 2). So in total, we have 4 MS runs for each cell line (A1, B1, A2, B2). In order to have confidence in the peptides we are seeing in our data, we ask that a peptide be seen in 3 out of the 4 total MS runs here. Our reasoning is that by asking for a 3/4 requirement, we will end up with peptides that were ID'd in both biological duplicates (A and B) at least once. Any missing values in these columns simply indicate that the peptide was not detected in that particular run. 

Column H: Log2R_Average
This one is just the average Log2 H/L ratio between the 4 MS runs associated with the experiment.

"Filtered_Out" sheet
This is the same as above, but all peptides here failed to meet the minimum requirements (i.e., only found in 1/4 or 2/4 runs). We are not interested in any of these peptides.

In [21]:
dpath = f"{DPATH_DATA}/{ID_IPYNB}/NCI60-isoDTB-SampleSet"
for dirpaths, dirnames, fns in os.walk(dpath):
    break
os.chdir(dpath)

# Starting from the root directory,
# read each organ directory,
# read the sheet "Final_data" from each excel file
organ2cellline2df= {}
for dirname in dirnames:
    os.chdir(dirname)
    ls_fp = glob(f"*.xlsx")
    if dirname not in organ2cellline2df:
        organ2cellline2df[dirname] = {}
    for i, fp in enumerate(ls_fp):
        df = pd.read_excel(fp, sheet_name="Final_data")
        df = df.set_index('Identifier')

        name = re.split('-|_', fp)[2]
        
        # filter out events (rows) with more than 1 non-NaN values,
        # so that 3/4 must have non-NaN values
        # use column iloc instead of name, because the names are too long.
        # Export with format # NCI60-[Name]-[Identifier]-Cys
        df = df[df.iloc[:, 2:6].isna().sum(axis=1) <=1]
        fp = f"{dpath}/{dirname}/NCI160-{name}-Cys.csv"
        df.to_csv(fp, header=True, index=True)

        # check if ratio is between (0.5, 1.5), same as log2 ration between (math.log(0.5, 2), math.log(1.5, 2))
        # admit only events with at least 3 ratios that fall between (0.5, 1.5)
        Stmp = df.iloc[:, 2:6].map(lambda x: (
            x > math.log(0.5, 2)) and (x < math.log(1.5, 2))).sum(axis=1)
        df = df[Stmp >= 3]

        # save the filtered df to the dictionary
        organ2cellline2df[dirname][i] = df
        
        # save the filtered df to a csv file under the same source folder
        fp = f"{dpath}/{dirname}/NCI160-{name}-HRCys.csv"
        print(fp)
        df.to_csv(fp, header=True, index=True)
        
    os.chdir(dpath)

# sanity check
organ2cellline2df.keys()

/gladstone/alexanian/datasets-online/VX_projects/Project_CycsteinProteomics/Experiment_CysteinEnrichment/Data/001/NCI60-isoDTB-SampleSet/Ovarian/NCI160-156-HRCys.csv
/gladstone/alexanian/datasets-online/VX_projects/Project_CycsteinProteomics/Experiment_CysteinEnrichment/Data/001/NCI60-isoDTB-SampleSet/Ovarian/NCI160-157-HRCys.csv
/gladstone/alexanian/datasets-online/VX_projects/Project_CycsteinProteomics/Experiment_CysteinEnrichment/Data/001/NCI60-isoDTB-SampleSet/Ovarian/NCI160-158-HRCys.csv
/gladstone/alexanian/datasets-online/VX_projects/Project_CycsteinProteomics/Experiment_CysteinEnrichment/Data/001/NCI60-isoDTB-SampleSet/Ovarian/NCI160-IGROV1-HRCys.csv
/gladstone/alexanian/datasets-online/VX_projects/Project_CycsteinProteomics/Experiment_CysteinEnrichment/Data/001/NCI60-isoDTB-SampleSet/Leukemia/NCI160-20240924-HRCys.csv
/gladstone/alexanian/datasets-online/VX_projects/Project_CycsteinProteomics/Experiment_CysteinEnrichment/Data/001/NCI60-isoDTB-SampleSet/Leukemia/NCI160-K562-HRC

dict_keys(['Ovarian', 'Leukemia'])

In [20]:
math.log(1.5, 2)

0.5849625007211562

In [9]:
organ2cellline2df['Leukemia'][0]

,AltIdentifier,Modified_Peptide,"Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240911_JLM191 - NCI60 isoDTB, CCRF-CEM, MOLT-4, and UACC-62/126-CCRFCEM/A1","Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240911_JLM191 - NCI60 isoDTB, CCRF-CEM, MOLT-4, and UACC-62/126-CCRFCEM/A2","Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240911_JLM191 - NCI60 isoDTB, CCRF-CEM, MOLT-4, and UACC-62/126-CCRFCEM/B1","Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240911_JLM191 - NCI60 isoDTB, CCRF-CEM, MOLT-4, and UACC-62/126-CCRFCEM/B2",Log2R_Average,Log2 Ratio HL (Median) (Count*)
Identifier,,,,,,,,
P05107_C472,ITGB2_C472,C*DTGYIGK,0.349855,0.493987,1.470249,0.580108,0.723550,4
Q92900_C683,UPF1_C683,QLILVGDHCQLGPVVMC*K,0.280768,0.060882,0.561343,1.916180,0.704793,4
Q14807_C72,KIF22_C72,GMDSC*SLEIANWR,0.549689,0.435109,0.354978,1.406956,0.686683,4
Q6P158_C1369,DHX57_C1369,IKNPSIDLCTC*PR,0.410887,0.563595,0.505190,1.045670,0.631336,4
Q8TCG2_C473,PI4K2B_C473,IVHLSNSFTQTVNC*R,0.521857,0.465191,0.565874,0.904159,0.614270,4
...,...,...,...,...,...,...,...,...
Q15649_C30,ZNHIT3_C30,VPYC*SVVCFR,-0.741258,-0.850510,-0.954385,-0.560345,-0.776625,4
Q02543_C64,RPL18A_C64,SSGEIVYC*GQVFEK,-0.901142,-0.828100,-0.534904,-0.868785,-0.783233,4
Q7Z6Z7_C1074,HUWE1_C1074,LC*VGSPVR,-0.942274,-0.854637,-0.807915,-0.567135,-0.792990,4


In [10]:
organ2cellline2df['Ovarian'][0]

,AltIdentifier,Modified_Peptide,"Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240816_JLM185 - NCI60 isoDTB, OVCAR-4 and OVCAR-5/156-OVCAR4/A1","Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240816_JLM185 - NCI60 isoDTB, OVCAR-4 and OVCAR-5/156-OVCAR4/A2","Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240816_JLM185 - NCI60 isoDTB, OVCAR-4 and OVCAR-5/156-OVCAR4/B1","Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240816_JLM185 - NCI60 isoDTB, OVCAR-4 and OVCAR-5/156-OVCAR4/B2",Log2R_Average
Identifier,,,,,,,
Q13614_C155,MTMR2_C155,GENSYGLETVC*K,0.494695,0.385324,0.574692,1.586510,0.760306
O95159_C16,ZFPL1_C16,VTNLFC*FEHR,0.381615,0.349463,0.512133,1.634914,0.719531
Q96R06_C589,SPAG5_C589,DAAEIVLEAFC*AHASQR,0.343944,0.533182,0.536115,1.447439,0.715170
O95749_C205,GGPS1_C205,SFC*EDLTEGK,0.481266,0.491256,1.105551,0.573407,0.662870
Q7Z2Z2_C953,EFL1_C953,GESPLTDC*YGPFSGQLIATMK,0.532048,0.515189,1.522865,0.037157,0.651815
...,...,...,...,...,...,...,...
Q6PL18_C1388,ATAD2_C1388,MEQEVENFSC*SR,-0.530976,-1.295350,-0.980429,-0.964268,-0.942756
Q6NUM6_C416,TYW1B_C416,HC*ALSLVGEPIMYPEINR,-0.995986,-0.869459,-0.816155,-1.104051,-0.946413
Q63ZY3_C249,KANK2_C249,SELC*LDLPDPPEDPVALETR,-1.213983,-0.880586,-0.933813,-0.988207,-1.004147


# filteration and normalization

In [11]:
for organ, cellline2df in organ2cellline2df.items():
    for cellline, df in cellline2df.items():
        print(organ, cellline, df.shape)
        
        break
    break

Ovarian 0 (393, 7)


In [12]:
df

,Identifier,AltIdentifier,Modified_Peptide,"Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240816_JLM185 - NCI60 isoDTB, OVCAR-4 and OVCAR-5/156-OVCAR4/A1","Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240816_JLM185 - NCI60 isoDTB, OVCAR-4 and OVCAR-5/156-OVCAR4/A2","Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240816_JLM185 - NCI60 isoDTB, OVCAR-4 and OVCAR-5/156-OVCAR4/B1","Log2R_/Users/jmontano/Documents/Rotations/Zaro Lab/Notebook/20240816_JLM185 - NCI60 isoDTB, OVCAR-4 and OVCAR-5/156-OVCAR4/B2",Log2R_Average
0,P00966_C97,ASS1_C97,YLLGTSLARPC*IAR,NaN,7.217590,8.508019,NaN,7.862804
1,Q69YN2_C511,CWF19L1_C511,QC*QISKEDEETLAR,NaN,NaN,NaN,6.792523,6.792523
2,Q7L5N1_C143,COPS6_C143,QVC*EIIESPLFLK,7.189342,NaN,6.811548,5.406283,6.469058
3,P49917_C371,LIG4_C371,RMVEDSDLQ[0.9840]TC*YCVFDVLMVNNK,NaN,NaN,NaN,6.461002,6.461002
4,Q5VW36_C1506,FOCAD_C1506,AASPLGSPELC*PSALHGLSQAMK,7.812624,NaN,2.270890,9.268659,6.450725
...,...,...,...,...,...,...,...,...
15552,Q92766_C648,RREB1_C648,FC*NQVFAFSGVLRAHVR,-5.732579,NaN,NaN,NaN,-5.732579
15553,O14647_C448,CHD2_C448,FQN[0.9840]C*IDSFHSR,NaN,-6.233219,NaN,NaN,-6.233219
15554,Q9ULJ7_C1116,ANKRD50_C1116,YGASSLN[0.9840]GC*SPSPVHTMEQKPLQSLSSK,-6.414922,NaN,NaN,NaN,-6.414922
15555,Q9Y5B9_C283,SUPT16H_C283,SYC*SNLVR,-6.820928,-6.204754,-6.468117,-6.284943,-6.444685
